In [0]:
import json
import os

# Clean up: Delete all existing .txt files and *_text_analysis.csv files from /dbfs/tmp
print("Cleaning up existing files...")
tmp_files = dbutils.fs.ls("dbfs:/tmp/")
for file_info in tmp_files:
    file_path = file_info.path
    if file_path.lower().endswith(".txt") or file_path.lower().endswith("_text_analysis.csv"):
        print(f"Deleting: {file_path}")
        dbutils.fs.rm(file_path)

# List .txt files in Unity Catalog volume
src_paths = [
    f.path for f in dbutils.fs.ls(dbutils.widgets.get("volume_path"))
    if f.path.lower().endswith(".txt")
]

copied_paths = []

# Copy each file to /dbfs/tmp and collect real local-accessible paths
for src in src_paths:
    filename = src.split("/")[-1]
    dst_java = f"/dbfs/tmp/{filename}"
    dst_spark = f"dbfs:/tmp/{filename}"
    dbutils.fs.cp(src, dst_spark, recurse=False)
    copied_paths.append(dst_java)  # Must be local path, NOT dbfs:/...

# Store for downstream use
dbutils.jobs.taskValues.set("file_paths", copied_paths)